## Url과 title 추출

In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 검색어, 검색 기간 정의
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

1.크롤링할 키워드는 무엇입니까?: 영화 리뷰


In [3]:
query_txt

'영화 리뷰'

In [4]:
# Step 1. 크롬 웹브라우저 실행
# pip install chromedriver_autoinstaller
import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
# driver : 크롬 브라우저

# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)  # 2초간 정지

In [5]:
# Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query") # css selector
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

In [6]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )
time.sleep(1)

# 검색옵션 확인
item_li = driver.find_elements_by_css_selector('.option .txt')  # class,id,name먼저 찾는게 베이직. id앞에는 #이붙는다.
item_li[3].text

'관련도순'

In [7]:
for i in range(0, len(item_li)):
    print(item_li[i].text)

전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기


In [8]:
item_li[10].click()

In [9]:
# 스크롤 다운
# driver.execute_script("window.scrollTo(0, 500)")
# time.sleep(2)

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 5  # 스크롤 1번 당 글 30개씩 화면에 보여짐
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행 (0,1,2,3,4)
    scroll_down(driver) # 스크롤 다운
    i = i+1

In [10]:
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit" # class는 맨앞에 점을 찍는다. 띄어쓰기로 되어있는 부분은.으로 연결
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

<selenium.webdriver.remote.webelement.WebElement (session="cdc4556074e7ea5c131ef103348d19b5", element="5da4cd2b-ac36-4daa-ac35-a4f294966d82")>

In [11]:
article_raw[0].get_attribute('href')

'https://blog.naver.com/pooh0512/222700777305'

In [12]:
# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')
    url_list.append(url)
time.sleep(1)

# 제목 크롤링 시작
for article in article_raw:
    title = article.text
    title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

[영화리뷰] 신비한 동물들과 덤블도어의 비밀 후기_'신동사...
[영화리뷰] 루이스 웨인: 사랑을 그린 고양이 화가 * 삶은...
[영화 리뷰/후기/쿠키] 넷플릭스 <야차> - 별 다른 특색은 없었던...
정보 - 집에서볼만한 한국 범죄 스릴러 추천 / 최민식 영화 리뷰
[영화 리뷰] 천국에서 무덤까지 - 삶의 빛을 잠식하는...
영화리뷰 나의 문어 선생님 넷플릭스 다큐 추천작
[영화리뷰]닥터 스트레인지 2
[영화리뷰] <범죄도시2> 후기 - 더위를 확실하게 날려줄 영화가...
영화 리뷰 민스미트 작전(출연진 평점) - 전쟁 실화 영화 추천
[영화리뷰] 닥터 스트레인지 2 : 대혼돈의 멀티버스 후기...
[영화리뷰] 닥터 스트레인지: 대혼돈의 멀티버스 * 멀티버스...
[영화 리뷰/후기/쿠키] <수퍼 소닉2> - 여전히 '짐 캐리' 없었으면...
[영화 리뷰] 패러렐 마더스 - 운명의 평행선 위에 선 여성들의...
쓰리: 아직 끝나지 않았다 최신 개봉 영화 리뷰 및 정보
[영화리뷰] <잭 인 더 박스> 후기 - 손잡이를 돌리면 시작되는...
[영화리뷰] 수퍼 소닉2 후기... 절대적 레벨 업 (최신 영화 추천)
[영화리뷰] 매스 * 분노와 아픔, 용서와 이해에 관한 이야기...
[영화 리뷰/후기/쿠키] <앵커> - 누군가의 잔혹 현대 심리극
[영화 리뷰] 불도저에 탄 소녀 - 자꾸 건드리는 세상을 향한...
[영화리뷰] 파리, 13구 * 사랑이 삶의 전부가 아니라 말해도...
[영화리뷰] <완벽한 축사를 준비하는 방법> 후기 - 인생은 보조...
[영화리뷰] 모비우스 후기_박쥐의 난 (쿠키)
[영화 리뷰/후기/쿠키] <폐교> - 이딴 영화 만드려고 학교 폭력을...
[영화 리뷰] 평평남녀 - 30대 여성의 일과 사랑의 이야기를 통해...
[영화리뷰] 더 노비스 * 채울 수 없는 목표를 위한 집착과...
[영화리뷰] 카시오페아 * 기억을 잃고, 그 과정을 지켜봐야 하는...
[영화리뷰] <서울괴담> 후기 - 10가지 괴담, 골라보는 재미가...
[영화리뷰] 범죄

In [13]:
# 수집된 url_list, title_list로 판다스 데이터프레임 만들기
df = pd.DataFrame({'url':url_list, 'title':title_list})
df  # 2차 구조(행,열)-> 엑셀구조  /  시리즈가 1차구조
# Data Frame에 url 리스트와 title 리스트를 저장함

,url,title
0,https://blog.naver.com/pooh0512/222700777305,[영화리뷰] 신비한 동물들과 덤블도어의 비밀 후기_'신동사...
1,https://blog.naver.com/kw1359/222696963331,[영화리뷰] 루이스 웨인: 사랑을 그린 고양이 화가 * 삶은...
2,https://blog.naver.com/duswns202/222695013360,[영화 리뷰/후기/쿠키] 넷플릭스 <야차> - 별 다른 특색은 없었던...
3,https://blog.naver.com/khd3001/222743553174,정보 - 집에서볼만한 한국 범죄 스릴러 추천 / 최민식 영화 리뷰
4,https://blog.naver.com/sgnam10/222687257204,[영화 리뷰] 천국에서 무덤까지 - 삶의 빛을 잠식하는...
...,...,...
175,https://blog.naver.com/lex958e9xe/222743765085,추억속 명작 스파이더맨1 영화리뷰.
176,https://blog.naver.com/myth408/222674461003,영화리뷰 옥자
177,https://blog.naver.com/zjvl851/222708415815,[영화리뷰] <니 부모 얼굴이 보고 싶다> - 자식이 괴물이 되면...
178,https://blog.naver.com/nyyii/222714074090,"[6언더그라운드] 영화 리뷰, 세상에 없는 히어로, 라이언 레이놀즈"


In [14]:
# 저장하기
df.to_csv("blog_url1.csv", encoding='utf-8-sig') # 한글 안깨지기위해서 utf-8-sig
# df.to_csv("blog_url.csv")

## 내용추출 및 저장

In [16]:
# "url_list.csv" 불러오기
url_load = pd.read_csv("blog_url1.csv")
url_load = url_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제 ,axis=1은 열

num_list = len(url_load)
print(num_list)
url_load

180


,url,title
0,https://blog.naver.com/pooh0512/222700777305,[영화리뷰] 신비한 동물들과 덤블도어의 비밀 후기_'신동사...
1,https://blog.naver.com/kw1359/222696963331,[영화리뷰] 루이스 웨인: 사랑을 그린 고양이 화가 * 삶은...
2,https://blog.naver.com/duswns202/222695013360,[영화 리뷰/후기/쿠키] 넷플릭스 <야차> - 별 다른 특색은 없었던...
3,https://blog.naver.com/khd3001/222743553174,정보 - 집에서볼만한 한국 범죄 스릴러 추천 / 최민식 영화 리뷰
4,https://blog.naver.com/sgnam10/222687257204,[영화 리뷰] 천국에서 무덤까지 - 삶의 빛을 잠식하는...
...,...,...
175,https://blog.naver.com/lex958e9xe/222743765085,추억속 명작 스파이더맨1 영화리뷰.
176,https://blog.naver.com/myth408/222674461003,영화리뷰 옥자
177,https://blog.naver.com/zjvl851/222708415815,[영화리뷰] <니 부모 얼굴이 보고 싶다> - 자식이 괴물이 되면...
178,https://blog.naver.com/nyyii/222714074090,"[6언더그라운드] 영화 리뷰, 세상에 없는 히어로, 라이언 레이놀즈"


In [17]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

# 수집할 글 갯수 정하기
number = 10

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)

    driver.get(url)   # 글 띄우기
    
    # 크롤링
    # 예외 처리
    try : 
        # iframe 접근(보통 네이버 사이트들에서 많이사용)
        driver.switch_to.frame('mainFrame')

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                                
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 글쓴이 크롤링
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title, nickname)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close() 
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        # print("에러",i, title)
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i==30 or i==50 or i==80:
        # 판다스 데이터프레임으로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_csv("blog_content1.csv", encoding='utf-8-sig')
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

  0%|          | 0/10 [00:00<?, ?it/s]

0 [영화리뷰] 신비한 동물들과 덤블도어의 비밀 후기_'신동사' 시리즈의 매력을 찾아야 할 때! (쿠키) 뀨우
1 [영화리뷰] 루이스 웨인: 사랑을 그린 고양이 화가 * 삶은 고통스러웠지만 그의 그림은 아름다웠기에 무비럽웅
2 [영화 리뷰/후기/쿠키] 넷플릭스 <야차> - 별 다른 특색은 없었던 귀신 야차의 미션임파서블 색시주뇨비
3 악마를 보았다 출연진 평점 정보 - 집에서볼만한 한국 범죄 스릴러 추천 / 최민식 영화 리뷰 피크
4 [영화 리뷰] 천국에서 무덤까지 - 삶의 빛을 잠식하는 트라우마의 어둠  리코짱
5 영화리뷰 나의 문어 선생님 넷플릭스 다큐 추천작 마시우
6 [영화리뷰]닥터 스트레인지 2  내일의죠
7 [영화리뷰] <범죄도시2> 후기 - 더위를 확실하게 날려줄 영화가 돌아왔다!  영소남
8 영화 리뷰 민스미트 작전(출연진 평점) - 전쟁 실화 영화 추천 제제
9 [영화리뷰] 닥터 스트레인지 2 : 대혼돈의 멀티버스 후기_멀티버스의 역습, 무엇이 현실인가? (4DX) 뀨우
수집한 글 갯수:  10
{0: {'title': "[영화리뷰] 신비한 동물들과 덤블도어의 비밀 후기_'신동사' 시리즈의 매력을 찾아야 할 때! (쿠키)", 'nickname': '뀨우', 'datetime': '2022. 4. 14. 17:16', 'content': '영화 : 신비한 동물들과 덤블도어의 비밀\n감독 : 데이빗 예이츠\n출연 : 에디 레드메인, 주드 로, 매즈 미켈슨, 에즈라 밀러\n관람일 : 2022년 4월 13일 (2022-42)\n개봉일 : 2022년 4월 13일 영화 <신비한 동물들과 덤블도어의 비밀> 리뷰\n영화 <신비한 동물들과 덤블도어의 비밀>을 극장 개봉 당일 관람하고 왔습니다. 140분이 훌쩍 넘는 러닝타임에다가 미리 관람한 분들이 지루하다는 평들이 많아서 내심 걱정을 하고 극장을 갔는데, 해덕으로 마법을 사랑하는 팬으로 충분히 반짝거리는 시간을 영화 내내 만날 수 있어 만족하고 나왔습니다. 하긴 이 시리즈가 산으로 간다는 평들이 많아도 저한테

In [18]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, orient='index')
result_df

,title,nickname,datetime,content
0,[영화리뷰] 신비한 동물들과 덤블도어의 비밀 후기_'신동사' 시리즈의 매력을 찾아야...,뀨우,2022. 4. 14. 17:16,영화 : 신비한 동물들과 덤블도어의 비밀\n감독 : 데이빗 예이츠\n출연 : 에디 ...
1,[영화리뷰] 루이스 웨인: 사랑을 그린 고양이 화가 * 삶은 고통스러웠지만 그의 그...,무비럽웅,2022. 4. 10. 21:45,루이스 웨인: 사랑을 그린 고양이 화가 \nThe Electrical Life o...
2,[영화 리뷰/후기/쿠키] 넷플릭스 <야차> - 별 다른 특색은 없었던 귀신 야차의 ...,색시주뇨비,2022. 4. 9. 7:54,<저의 리뷰는 관점에 따라 약간의 스포일러가 있을 수 있습니다.> [영화리뷰/후기/...
3,악마를 보았다 출연진 평점 정보 - 집에서볼만한 한국 범죄 스릴러 추천 / 최민식 ...,피크,17시간 전,악마를 보았다 출연진 평점 정보입니다 집에서볼만한 한국 범죄 스릴러로 추천하며 최민...
4,[영화 리뷰] 천국에서 무덤까지 - 삶의 빛을 잠식하는 트라우마의 어둠,리코짱,2022. 3. 30. 19:58,[영화 리뷰] 천국에서 무덤까지 - 삶의 빛을 잠식하는 트라우마의 어둠 영화 줄거...
5,영화리뷰 나의 문어 선생님 넷플릭스 다큐 추천작,마시우,2022. 3. 11. 12:51,오늘은 넷플릭스 다큐 《 나의 문어 선생님 》 영화리뷰와 관련된 이야기를 나눌까 합...
6,[영화리뷰]닥터 스트레인지 2,내일의죠,2022. 5. 9. 0:10,[영화리뷰]\n닥터 스트레인지 2 대혼돈의 멀티버스를 보고 왔습니다.\n스파이더맨 ...
7,[영화리뷰] <범죄도시2> 후기 - 더위를 확실하게 날려줄 영화가 돌아왔다!,영소남,2022. 5. 13. 19:10,"안녕하세요, 영소남입니다. 688만 명을 극장으로 불러들이며 역대 청불 영화 TO..."
8,영화 리뷰 민스미트 작전(출연진 평점) - 전쟁 실화 영화 추천,제제,2022. 5. 11. 17:46,#영화리뷰 #민스미트작전 #민스미트작전출연진 #전쟁영화추천 #최신개봉영화 #역사영화...
9,"[영화리뷰] 닥터 스트레인지 2 : 대혼돈의 멀티버스 후기_멀티버스의 역습, 무엇이...",뀨우,2022. 5. 7. 14:08,영화 : 닥터 스트레인지: 대혼돈의 멀티버스\n감독 : 샘 레이미\n출연 : 베네딕...


In [19]:
# 엑셀로 저장하기
result_df.to_csv("blog_content1.csv", encoding='utf-8-sig')